In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot.tf_keras import PlotLossesCallback
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.metrics import confusion_matrix
from sklearn import metrics

import numpy as np
np.random.seed(42)
import warnings;warnings.simplefilter('ignore')
%matplotlib inline
print('Tensorflow version:', tf.__version__)

Tensorflow version: 2.12.0


In [ ]:
train_images = pd.read_csv('dataset/train/images.csv',header=None)
train_labels = pd.read_csv('dataset/train/labels.csv',header=None)
validation_images = pd.read_csv('dataset/validation/images.csv',header=None)
validation_labels = pd.read_csv('dataset/validation/labels.csv',header=None)

Load and Preprocess Data

In [ ]:
train_images.head()

,0,1,2,3,4,5,6,7,8,9,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,0.631373,0.623529,0.713726,0.705882,0.658824,0.666667,0.654902,0.635294,0.647059,0.705882,...,0.682353,0.611765,0.650980,0.658824,0.600000,0.603922,0.654902,0.694118,0.658824,0.666667
1,0.725490,0.752941,0.749020,0.701961,0.690196,0.721569,0.709804,0.745098,0.654902,0.721569,...,0.721569,0.698039,0.721569,0.686275,0.713726,0.682353,0.690196,0.698039,0.701961,0.725490
2,0.717647,0.701961,0.713726,0.733333,0.705882,0.717647,0.725490,0.682353,0.717647,0.674510,...,0.709804,0.694118,0.705882,0.682353,0.639216,0.713726,0.670588,0.678431,0.737255,0.674510
3,0.705882,0.674510,0.654902,0.678431,0.666667,0.662745,0.678431,0.662745,0.686275,0.686275,...,0.639216,0.662745,0.631373,0.643137,0.705882,0.662745,0.705882,0.666667,0.654902,0.631373
4,0.647059,0.729412,0.701961,0.674510,0.611765,0.698039,0.713726,0.662745,0.701961,0.674510,...,0.639216,0.670588,0.705882,0.674510,0.721569,0.694118,0.674510,0.705882,0.749020,0.729412


In [ ]:
train_labels.head()
#0 == squigle
#1 == narrow band
#2 == nOise
#3 == narrow band drd

,0,1,2,3
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [ ]:
print("Training set shape: ", train_images.shape, train_labels.shape)
print("Validation set shape: ", validation_images.shape, validation_labels.shape)

In [ ]:
x_train = train_images.values.reshape(3200,64,128,1)
x_val = validation_images.values.reshape(800,64,128,1)

y_train = train_labels.values
y_val = validation_labels.values

Plotting 2D Spectrograms

In [ ]:
plt.figure(0,figsize=(12,12))
for i in range(1,10):
    plt.subplot(3,3,i)
    image = np.squeeze(x_train[np.random.randint(0,3200)])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image,cmap='gray')

In [ ]:
plt.figure(0,figsize=(12,12))
for i in range(1,4):
    plt.subplot(1,3,i)
    image = np.squeeze(x_train[np.random.randint(0,3200)])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image,cmap='jet')

Create Training and Validation Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(horizontal_flip=True)
datagen_train.fit(x_train)
datagen_val = ImageDataGenerator(horizontal_flip=True)
datagen_val.fit(x_val)

Creating the CNN Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialising the CNN
model = Sequential()
# 1st Convolution
model.add(Conv2D(32,(5,5),padding='same',input_shape=(64,128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# 2nd Convolution layer
model.add(Conv2D(64,(5,5),padding='same',input_shape=(64,128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Fully connected layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
# final prediction layer
model.add(Dense(4,activation='softmax'))

Learning Rate Scheduling and Compiling the model

In [ ]:
initial_learning  = 0.005
lr_schedule =  tf.keras.optimizers.schedules.ExponentialDecay( initial_learning_rate = initial_learning,decay_rate = 0.96,decay_steps = 5, staircase = True)
optimizer = Adam(learning_rate = lr_schedule)

In [ ]:
model.compile(optimizer= optimizer,metrics=['accuracy'],loss='categorical_crossentropy')
model.summary()

Training the Model

In [ ]:
checkpoint = ModelCheckpoint('weights.h5',monitor = 'val_loss',save_weights_only = True, mode= 'min', verbose = 0)
callbacks = [PlotLossesCallback,checkpoint]
batch_size = 32
history = model.fit_generator(datagen_train.flow(x_train,y_train,shuffle=True,batch_size=batch_size),
                   steps_per_epoch = len(x_train)//batch_size,
                   validation_data= datagen_val.flow(x_val,y_val,shuffle=True,batch_size=batch_size),
                   validation_steps = len(x_val)//batch_size,
                   epochs=10
                   callbacks=[checkpoint]
                   )

Evaluating the Model

In [ ]:
model.evaluate(x_val,y_val)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

y_true = np.argmax(y_val,1)
y_pred = np.argmax(model.predict(x_val),1)
print(metrics.classification_report(y_true,y_pred))
print("Classification Accuracy: %0.6f" %metrics.accuracy_score(y_true,y_pred))

In [ ]:
labels = ["squiggle", "narrowband", "noise", "narrowbanddrd"]
ax= plt.subplot()
sns.heatmap(metrics.confusion_matrix(y_true,y_pred,normalize='true'),annot=True,ax=ax,cmap=plt.cm.Blues);

ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(labels);
ax.yaxis.set_ticklabels(labels);